In [1]:
import torch

In [8]:
from models import LAMRGModel_v12

In [9]:
from modules.tokenizers import Tokenizer

In [10]:
from argparse import Namespace
args = Namespace(image_dir='data/mimic/r2gen/images/images224/files', ann_path='data/mimic/r2gen/images/images224/annotation.json', label_path='data/mimic/finding/chexpert_labeled.csv', image_size=256, crop_size=224, dataset_name='mimic_cxr', max_seq_length=80, threshold=3, num_workers=4, batch_size=16, version='12', visual_extractor='resnet101', visual_extractor_pretrained=True, pretrain_cnn_file=None, d_model=512, d_ff=512, d_vf=2048, num_heads=8, num_memory_heads=8, num_layers=3, num_labels=14, dropout=0.1, logit_layers=1, bos_idx=0, eos_idx=0, pad_idx=0, use_bn=0, drop_prob_lm=0.1, sample_method='beam_search', beam_size=3, temperature=1.0, sample_n=1, group_size=1, output_logsoftmax=1, decoding_constraint=0, block_trigrams=1, n_gpu=1, epochs=30, save_dir='results\\mimic_cxr\\V12_resnet101_labelloss_rankloss_10th_try_balanced_train_20240329-235144', expe_name='10th_try_balanced_train', record_dir='records/', save_period=1, monitor_mode='max', monitor_metric='BLEU_4', early_stop=50, label_smoothing=0.1, alpha=0.5, grad_clip=5, test_steps=500, label_loss=True, rank_loss=True, optim='Adam', lr_ve=5e-05, lr_ed=0.0001, weight_decay=5e-05, amsgrad=True, lr_scheduler='StepLR', step_size=1, gamma=0.8, seed=456789, gpu='0', resume=None, num_slots=60, rm_num_heads=8, rm_d_model=512, cfg='config/mimic_resnet.yml', set_cfgs=[])

In [11]:
tokenizer = Tokenizer(args)
model = LAMRGModel_v12(args, tokenizer)

=> creating model 'resnet101'


C:\Users\super\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\super\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Load pretrained CNN model from: official pretrained in ImageNet


In [14]:
import os
best = torch.load(os.path.join('results/mimic_cxr/V12_resnet101_labelloss_rankloss_10th_try_balanced_train_20240329-235144/model_best.pth'))

In [19]:
best.keys()

dict_keys(['epoch', 'state_dict', 'optimizer', 'monitor_best'])

In [20]:
model.load_state_dict(best['state_dict'])

<All keys matched successfully>

In [22]:
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('best_mimic_model_scripted.pt') # Save

<unknown>:6: SyntaxWarning: invalid escape sequence '\['
<unknown>:10: SyntaxWarning: invalid escape sequence '\['
<unknown>:40: SyntaxWarning: invalid escape sequence '\['
<unknown>:55: SyntaxWarning: invalid escape sequence '\['
<unknown>:35: SyntaxWarning: invalid escape sequence '\['
<unknown>:50: SyntaxWarning: invalid escape sequence '\['
<unknown>:6: SyntaxWarning: invalid escape sequence '\['
<unknown>:6: SyntaxWarning: invalid escape sequence '\['
<unknown>:10: SyntaxWarning: invalid escape sequence '\['
<unknown>:40: SyntaxWarning: invalid escape sequence '\['
<unknown>:55: SyntaxWarning: invalid escape sequence '\['
<unknown>:35: SyntaxWarning: invalid escape sequence '\['
<unknown>:50: SyntaxWarning: invalid escape sequence '\['
<unknown>:6: SyntaxWarning: invalid escape sequence '\['
<unknown>:6: SyntaxWarning: invalid escape sequence '\['
<unknown>:10: SyntaxWarning: invalid escape sequence '\['
<unknown>:40: SyntaxWarning: invalid escape sequence '\['
<unknown>:55: Synta

RuntimeError: 
Module 'VisualExtractor' has no attribute 'args' (This attribute exists on the Python module, but we failed to convert Python type: 'argparse.Namespace' to a TorchScript type. Only tensors and (possibly nested) tuples of tensors, lists, or dictsare supported as inputs or outputs of traced functions, but instead got value of type Namespace.. Its type was inferred; try adding a type annotation for the attribute.):
  File "D:\Py\M2KT\modules\visual_extractor.py", line 78
    def forward(self, images):
        if self.args.visual_extractor == 'densenet':
           ~~~~~~~~~ <--- HERE
            patch_feats = self.model.densenet121.features(images)
            avg_feats = self.avg_fnt(patch_feats).squeeze().reshape(-1, patch_feats.size(1))
